## Deliverable 2. Create a Customer Travel Destinations Map.

In [22]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

In [23]:
# Import API key
from config import g_key

In [24]:
# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [26]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Jamestown,US,42.0970,-79.2353,76.17,59,1,1.99,clear sky
1,1,Ushuaia,AR,-54.8000,-68.3000,38.82,70,40,0.00,scattered clouds
2,2,Swan River,CA,52.1058,-101.2676,83.71,29,23,19.42,few clouds
3,3,Bethel,US,41.3712,-73.4140,88.02,74,75,4.00,broken clouds
4,4,Nanortalik,GL,60.1432,-45.2371,50.25,90,100,8.28,overcast clouds


In [27]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [28]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Jamestown,US,42.0970,-79.2353,76.17,59,1,1.99,clear sky
2,2,Swan River,CA,52.1058,-101.2676,83.71,29,23,19.42,few clouds
3,3,Bethel,US,41.3712,-73.4140,88.02,74,75,4.00,broken clouds
7,7,Marienburg,PL,54.0359,19.0266,79.84,38,71,8.84,broken clouds
13,13,Albany,US,42.6001,-73.9662,81.48,66,79,4.00,broken clouds
25,25,Parbhani,IN,19.2667,76.7833,81.21,62,79,6.13,broken clouds
29,29,Hamilton,US,39.1834,-84.5333,81.88,64,1,1.99,clear sky
32,32,Komsomolskiy,UZ,40.4272,71.7189,79.52,27,21,0.00,few clouds
36,36,Ponta Do Sol,PT,32.6667,-17.1000,83.71,71,83,1.21,broken clouds
39,39,Harper,LR,4.3750,-7.7169,79.92,60,80,7.18,broken clouds


In [29]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                203
City                   203
Country                203
Lat                    203
Lng                    203
Max Temp               203
Humidity               203
Cloudiness             203
Wind Speed             203
Current Description    203
dtype: int64

In [33]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df

In [34]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Jamestown,US,76.17,clear sky,42.0970,-79.2353,
2,Swan River,CA,83.71,few clouds,52.1058,-101.2676,
3,Bethel,US,88.02,broken clouds,41.3712,-73.4140,
7,Marienburg,PL,79.84,broken clouds,54.0359,19.0266,
13,Albany,US,81.48,broken clouds,42.6001,-73.9662,
25,Parbhani,IN,81.21,broken clouds,19.2667,76.7833,
29,Hamilton,US,81.88,clear sky,39.1834,-84.5333,
32,Komsomolskiy,UZ,79.52,few clouds,40.4272,71.7189,
36,Ponta Do Sol,PT,83.71,broken clouds,32.6667,-17.1000,
39,Harper,LR,79.92,broken clouds,4.3750,-7.7169,


In [35]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [36]:
# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [43]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"]!='')]
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Jamestown,US,76.17,clear sky,42.0970,-79.2353,Hampton Inn & Suites Jamestown
2,Swan River,CA,83.71,few clouds,52.1058,-101.2676,Super 8 by Wyndham Swan River MB
3,Bethel,US,88.02,broken clouds,41.3712,-73.4140,Courtyard by Marriott Danbury
7,Marienburg,PL,79.84,broken clouds,54.0359,19.0266,Hotel Centrum Malbork
25,Parbhani,IN,81.21,broken clouds,19.2667,76.7833,City Palace
...,...,...,...,...,...,...,...
550,Conde,BR,83.66,scattered clouds,-7.2597,-34.9075,Pousada Beija Flor
551,Bachaquero,VE,84.52,overcast clouds,9.9694,-71.1494,casa de los dinosaurios
552,Valleyview,CA,77.04,broken clouds,55.0667,-117.2859,Paradise Inn & Suites
553,Maues,BR,86.90,overcast clouds,-3.3836,-57.7186,Hotel Miramar


In [45]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [54]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [55]:
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [58]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [59]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [60]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]


In [61]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))